In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [3]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [4]:
question="what is AI "

In [5]:
print(llm.invoke(question).content)

**Artificial Intelligence (AI)**

AI refers to the development and application of computer systems that are capable of performing tasks that normally require human intelligence, such as:

* **Learning and Adaptation:** AI systems can learn from data, identify patterns, and adapt to changing circumstances.
* **Reasoning and Problem-Solving:** AI systems can apply logical reasoning and solve problems effectively.
* **Natural Language Processing:** AI systems can understand, generate, and communicate in human language.
* **Image and Speech Recognition:** AI systems can analyze images and speech to extract meaningful information.
* **Decision-Making:** AI systems can make informed decisions based on available data and knowledge.

**Types of AI:**

* **Machine Learning:** AI systems that learn from data and improve their performance over time.
* **Deep Learning:** A subset of machine learning that uses neural networks to learn from large amounts of data.
* **Natural Language Processing (NLP

### **Simple AI Assistant**

In [6]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [7]:
store={}

In [8]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [9]:
config = {"configurable": {"session_id": "firstchat"}}

In [10]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [11]:
model_with_memory.invoke(("Hi! I'm Sureshbeekhani"),config=config).content

"Hi Sureshbeekhani,\n\nI'm Gemini, a multi-modal AI language model developed by Google, and I'm happy to help you with whatever you need.\n\nIs there anything specific I can assist you with today?"

In [12]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Sureshbeekhani.\n\nIs there anything else I can help you with today?'

In [13]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Sureshbeekhani", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Sureshbeekhani,\n\nI'm Gemini, a multi-modal AI language model developed by Google, and I'm happy to help you with whatever you need.\n\nIs there anything specific I can assist you with today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-16fb940e-ddcd-45c7-bbde-3adeb233cc05-0', usage_metadata={'input_tokens': 9, 'output_tokens': 47, 'total_tokens':

### **RAG with LCEL**

In [14]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [15]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [17]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [18]:
question ="what is llama?"
print(retrieval_chain.invoke(question))

LLaMA is a family of language models developed by Meta AI.


### **Let's Start with Tools and Agents**

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper=WikipediaAPIWrapper()

In [21]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [22]:
tool.name

'wikipedia'

In [23]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [24]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [25]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment informa

In [26]:
from langchain_community.tools import YouTubeSearchTool

In [27]:
tool2=YouTubeSearchTool()

In [28]:
tool2.name

'youtube_search'

In [29]:
tool2.run("Irfan Malik")

"['https://www.youtube.com/watch?v=YIOfTrhEdEg&pp=ygULSXJmYW4gTWFsaWs%3D', 'https://www.youtube.com/watch?v=R0L9skF9NBw&pp=ygULSXJmYW4gTWFsaWs%3D']"

In [30]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [31]:
tool3=TavilySearchResults()

In [34]:
tool.invoke({"query": "What happened in the latest burning man floods"})

'Page: Burning Man\nSummary: Burning Man is a week-long large-scale desert event focused on "community, art, self-expression, and self-reliance" held annually in the western United States. The event\'s name comes from its culminating ceremony: the symbolic burning of a large wooden effigy, referred to as the Man, that occurs on the penultimate night, the Saturday evening before Labor Day. Since 1990, the event has been at Black Rock City in northwestern Nevada, a temporary city erected in the Black Rock Desert about 100 miles (160 km) north-northeast of Reno. According to Burning Man co-founder Larry Harvey in 2004, the event is guided by ten stated principles: radical inclusion, gifting, decommodification, radical self-reliance, radical self-expression, communal effort, civic responsibility, leaving no trace, participation, and immediacy.\nBurning Man features no headliners or scheduled performers; participants create all the art, activities, and events. Artwork includes experimental 

In [35]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [37]:
tool=load_tools(["wikipedia"],llm=llm)

In [38]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_11536\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [39]:
agent.run("What is current GDP of pakistan?")

C:\Users\SURESH BEEKHANI\AppData\Local\Temp\ipykernel_11536\3358457773.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is current GDP of pakistan?")




> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: GDP of Pakistan
Observation: Page: Economy of Pakistan
Summary: The economy of Pakistan is categorized as a developing economy. It ranks as the 24th-largest based on GDP using purchasing power parity (PPP) and the 43rd largest in terms of nominal GDP. With a population of 254.4 million people as of 2024, Pakistan's position at per capita income ranks 161st by GDP (nominal) and 138th by GDP (PPP) according to the International Monetary Fund (IMF).
In its early years, Pakistan's economy relied heavily on private industries. The nationalization of a significant portion of the sector, including financial services, manufacturing, and transportation, began in the early 1970s under Zulfikar Ali Bhutto. During Zia-ul Haq's regime in the 1980s, an "Islamic" economy was adopted, outlawing economic practices forbidden in Sharīʿah and mandating traditional religious practices. The economy started privatizing again in the 1990s

"I'm sorry, I cannot answer this question as the GDP data is not available in the context."